# Subgroup Discovery in Python

В данном блокноте мы рассмотрим, как можно использовать Subgroup Discovery.
Но прежде чем мы начнём необходмо установить пакет pysubgroup.

```console
pip install pysubgroup 
```


In [1]:
import numpy as np
import pandas as pd
import pysubgroup as ps

In [2]:
# Только для красоты, в реальной жизни Warnings могут помочь найти ошибку в коде
import warnings
warnings.filterwarnings('ignore')

## Загрузка данных

Будем использовать данные с [Титаника, Kaggle InClass](https://www.kaggle.com/c/titanic).

In [3]:
db = pd.read_csv("./SD/data/train.csv")

In [4]:
print(db.head())
print(db.info())
print(db.describe())
print(db.isna().sum())

   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S  
<c

In [5]:
# Убираем неопределённые значения
db.Embarked.fillna("S", inplace=True) # Только 2 таких, обощить их нельзя
db["IsAgeNA"] = db.Age.isna() # А тут таких много, вдруг есть закономерность. Сохраним.
db.Age.fillna(db.Age.mean(), inplace=True) # Средний возраст, почему бы и нет

In [6]:
catVars = ["PassengerId","Survived","Pclass","Sex","Cabin","Embarked"]
db[catVars] = db[catVars].astype("category")
db.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 13 columns):
PassengerId    891 non-null category
Survived       891 non-null category
Pclass         891 non-null category
Name           891 non-null object
Sex            891 non-null category
Age            891 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null category
Embarked       891 non-null category
IsAgeNA        891 non-null bool
dtypes: bool(1), category(6), float64(2), int64(2), object(2)
memory usage: 103.2+ KB


In [7]:
db = db.drop(columns=["PassengerId","Name","Ticket","Cabin"])
db.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
Survived    891 non-null category
Pclass      891 non-null category
Sex         891 non-null category
Age         891 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Fare        891 non-null float64
Embarked    891 non-null category
IsAgeNA     891 non-null bool
dtypes: bool(1), category(4), float64(2), int64(2)
memory usage: 32.7 KB


## Бизнес задача "Кто выживал на Титанике"

Мы хотим понять, кто выживал и кто не выживал на Титанике. Назовём это фундаментальным исследованием социальной картины выживания на титанике.

Сначала нам нужно определить через какую (какие) переменную задаётся задача. В данном случае это survived, бинарная переменная.

In [8]:
target = ps.NominalTarget(target_selector=ps.NominalSelector('Survived', 1))
# 1 - выживший, т.е. тут мы ищем подгруппы связанные с выжившими

Далее нам нужно определить пространство поиска, т.е. тот набор "вопросов", которые будут "задаваться" данным для определения подгрупп. Возмем все переменные кроме 'survived'.

In [9]:
searchSpace = ps.create_selectors(db, ignore=['Survived'])

Ну и создаём задачу, которую затем и исполняем

In [10]:
task = ps.SubgroupDiscoveryTask(
    db,
    target=target,
    search_space=searchSpace,
    result_set_size = 5, # на выход получаем 5 подгрупп
    depth = 3, # Каждая подгруппа описывается не более чем 3 предикатами
    qf = ps.StandardQF(1) # Функция качества на базе Chi2
)

result = ps.BeamSearch().execute(task)

In [11]:
for (q,sg) in result:
    print(str(q) + ":\t" + str(sg.subgroup_description))

0.12623428448344273:	Sex=female
0.10866238138965413:	Sex=female AND IsAgeNA=False
0.08814293326077838:	Sex=female AND Parch: [0:1[
0.07880148284188689:	Sex=female AND SibSp: [0:1[
0.07105850876894648:	Sex=female AND Embarked=S


А если исключить пол?

In [12]:
searchSpace = ps.create_selectors(db, ignore=['Survived','Sex'])
task = ps.SubgroupDiscoveryTask(
    db,
    target=target,
    search_space=searchSpace,
    result_set_size = 5, # на выход получаем 5 подгрупп
    depth = 3, # Каждая подгруппа описывается не более чем 3 предикатами
    qf = ps.StandardQF(1) # Функция качества на базе Chi2
)
result = ps.BeamSearch().execute(task)

for (q,sg) in result:
    print(str(q) + ":\t" + str(sg.subgroup_description))

0.05958575655545354:	Pclass=1
0.05679692548379417:	Pclass=1 AND IsAgeNA=False
0.05382670702536022:	Pclass=1 AND Fare>=39.69
0.049756827534605315:	Fare>=39.69 AND IsAgeNA=False AND Pclass=1
0.0484190955571427:	Fare>=39.69


В целом видим, что выживаемость выше в первом у женщин и в первом классе.

## Бизнес задача "Предсказательная модель для Титаника"

В этом разделе мы рассмотрим как методы класса SD могут применятся для улучшения предсказательных моделей.

Первым делом строим предсказательную модель.

In [13]:
from sklearn.linear_model import LogisticRegression

Определим функции обучения и предсказания

In [14]:
def TrainModel(X, y):
    X_ohe = pd.get_dummies(X)
    return LogisticRegression(random_state=0,solver="lbfgs").fit(X_ohe,y)
def PredictByModel(m, X):
    X_ohe = pd.get_dummies(X)
    return m.predict(X_ohe)

Обучимся и проверим где у нас ошибки?

In [15]:
X = db.drop(columns = ["Survived"])
m = TrainModel(X,db.Survived)
db_sg=db.copy()
db_sg["error"] = PredictByModel(m,X) != db.Survived
db_sg.error.describe()

count       891
unique        2
top       False
freq        712
Name: error, dtype: object

Давайте попробуем описать наблюдения, в которых у нас ошибки с помощью методов SD.

In [16]:
target = ps.NominalTarget(target_selector=ps.NominalSelector('error', True))
# Интересует именно ошибки, поэтому True
searchSpace = ps.create_selectors(db_sg, ignore=['error']) 
# Технически есть ещё Survived, но он нам может подсказать, в каком направлении происходит ошибка, поэтому оставляем
task = ps.SubgroupDiscoveryTask(
    db_sg,
    target=target,
    search_space=searchSpace,
    result_set_size = 5, # на выход получаем 5 подгрупп
    depth = 3, # Каждая подгруппа описывается не более чем 3 предикатами
    qf = ps.StandardQF(1) # Функция качества на базе Chi2
)

result = ps.BeamSearch().execute(task)

In [17]:
for (q,sg) in result:
    print(str(q) + ":\t" + str(sg.subgroup_description))

0.08092270755944532:	Survived=1 AND Sex=male
0.06769528430583426:	Survived=1 AND Sex=male AND IsAgeNA=False
0.06569120560890108:	Survived=1 AND Sex=male AND Embarked=S
0.06277011290105192:	Survived=1 AND Sex=male AND Parch: [0:1[
0.05312509053623906:	Survived=1 AND Sex=male AND SibSp: [0:1[


In [18]:
db_sg[["Survived","Sex","error"]].groupby(['Survived',"Sex"]).agg([np.size,np.sum,np.mean])

error                
                 size   sum      mean
Survived Sex                         
0        female    81  60.0  0.740741
         male     468  15.0  0.032051
1        female   233  10.0  0.042918
         male     109  94.0  0.862385

### Улучшаем модель

Модель плохо работает на мужчинах и женщинах на разных классах. Видимо нужна нелиненость. Пусть будет две модели.

In [19]:
def TrainModel(X, y):
    isMale = (X.Sex=="male")
    return { "Male": LogisticRegression(random_state=0,solver="lbfgs").
                fit(pd.get_dummies(X[isMale].drop(columns=["Sex"])),y[isMale]),
            "Female":  LogisticRegression(random_state=0,solver="lbfgs").
                fit(pd.get_dummies(X[np.invert(isMale)].drop(columns=["Sex"])),y[np.invert(isMale)])}
def PredictByModel(m, X):
    sex = X.Sex
    X_ohe = pd.get_dummies(X.drop(columns=["Sex"]))
    ans = m["Male"].predict(X_ohe)
    ans[sex == "female"] = m["Female"].predict(X_ohe[sex=="female"])
    return ans

In [20]:
X = db.drop(columns = ["Survived"])
m = TrainModel(X,db.Survived)
db_sg=db.copy()
db_sg["error"] = PredictByModel(m,X) != db.Survived
db_sg.error.describe()

count       891
unique        2
top       False
freq        731
Name: error, dtype: object

Давайте попробуем описать наблюдения, в которых у нас ошибки с помощью методов SD.

In [21]:
target = ps.NominalTarget(target_selector=ps.NominalSelector('error', True))
# Интересует именно ошибки, поэтому True
searchSpace = ps.create_selectors(db_sg, ignore=['error']) 
# Технически есть ещё Survived, но он нам может подсказать, в каком направлении происходит ошибка, поэтому оставляем
task = ps.SubgroupDiscoveryTask(
    db_sg,
    target=target,
    search_space=searchSpace,
    result_set_size = 5, # на выход получаем 5 подгрупп
    depth = 3, # Каждая подгруппа описывается не более чем 3 предикатами
    qf = ps.StandardQF(1) # Функция качества на базе Chi2
)

result = ps.BeamSearch().execute(task)

In [22]:
for (q,sg) in result:
    print(str(q) + ":\t" + str(sg.subgroup_description))

0.08353141087896046:	Survived=1 AND Sex=male
0.06879872424204636:	Survived=1 AND Sex=male AND IsAgeNA=False
0.06692942645056375:	Survived=1 AND Parch: [0:1[ AND Sex=male
0.06641171661747794:	Survived=1 AND Embarked=S AND Sex=male
0.05575016910594913:	Survived=1 AND Embarked=S


In [23]:
db_sg[["Survived","Sex","error"]].groupby(['Survived',"Sex"]).agg([np.size,np.sum,np.mean])

error                
                 size   sum      mean
Survived Sex                         
0        female    81  40.0  0.493827
         male     468  10.0  0.021368
1        female   233  16.0  0.068670
         male     109  94.0  0.862385

Ситуация улучшилась, но всё ещё проблемы с полом. С нужно для мужчин думать о новых переменных.
Давайте посмотрим на другие переменные, т.е. теперь мы не хотим выделять подгруппы по полу.
Убираем по очереди переменные 'Sex','Pclass','Embarked','Survived' из рассмотрения.

In [24]:
target = ps.NominalTarget(target_selector=ps.NominalSelector('error', True))
# Интересует именно ошибки, поэтому True
searchSpace = ps.create_selectors(db_sg, ignore=['error','Sex','Pclass','Embarked','Survived']) 
# Технически есть ещё Survived, но он нам может подсказать, в каком направлении происходит ошибка, поэтому оставляем
task = ps.SubgroupDiscoveryTask(
    db_sg,
    target=target,
    search_space=searchSpace,
    result_set_size = 5, # на выход получаем 5 подгрупп
    depth = 3, # Каждая подгруппа описывается не более чем 3 предикатами
    qf = ps.StandardQF(1) # Функция качества на базе Chi2
)

result = ps.BeamSearch().execute(task)

In [25]:
for (q,sg) in result:
    print(str(q) + ":\t" + str(sg.subgroup_description))

0.01504507602524812:	Fare: [10.50:21.68[ AND SibSp: [1:2[
0.01287220628784415:	SibSp: [1:2[
0.012326784492889995:	Fare: [10.50:21.68[
0.011622648734508068:	Fare: [10.50:21.68[ AND Parch: [1:2[
0.011378279616214525:	SibSp: [1:2[ AND IsAgeNA=False


## Бизнес задача "Клиенты и воздействие"

Компания делает рассылку и нужно понять как работало воздействие.

In [26]:
db = pd.read_csv("http://www.minethatdata.com/Kevin_Hillstrom_MineThatData_E-MailAnalytics_DataMiningChallenge_2008.03.20.csv")
db[['history_segment','zip_code','channel','segment']]= \
    db[['history_segment','zip_code','channel','segment']].astype('category')
work_db=db.loc[(db.segment == "Womens E-Mail") | (db.segment == "No E-Mail")]


In [27]:
db_sg = work_db.drop(columns=["segment","visit","conversion","spend"])
y = work_db.visit
trt = work_db.segment=="Womens E-Mail"
Z = y*trt + (1-y)*(1-trt)
db_sg["Z"]=Z

In [28]:
target = ps.NominalTarget(target_selector=ps.NominalSelector('Z', 0))
# Интересует именно ошибки, поэтому True
searchSpace = ps.create_selectors(db_sg, ignore=['Z',]) 
# Технически есть ещё Survived, но он нам может подсказать, в каком направлении происходит ошибка, поэтому оставляем
task = ps.SubgroupDiscoveryTask(
    db_sg,
    target=target,
    search_space=searchSpace,
    result_set_size = 5, # на выход получаем 5 подгрупп
    depth = 3, # Каждая подгруппа описывается не более чем 3 предикатами
    qf = ps.ChiSquaredQF() # Функция качества на базе Chi2
)

result = ps.BeamSearch().execute(task)

for (q,sg) in result:
    print(str(q) + ":\t" + str(sg.subgroup_description))

36.2050279685305:	womens=1
36.2050279685305:	womens=0
36.2050279685305:	mens=1 AND womens=0
22.2275079185997:	mens=0 AND womens=1
22.2275079185997:	mens=0


In [29]:
print(trt.mean())
print(db_sg.Z.mean())
db_sg[["womens","mens","Z"]].groupby(["womens","mens"]).agg([np.mean,np.size])

0.5009486332654065
0.5219122572787108


Z       
                 mean   size
womens mens                 
0      1     0.505867  19260
1      0     0.534540  19166
       1     0.537614   4267

Получаем, что акция релевантна только тем, кто уже покупал женский товары. Это неудивительно, мы ведь рекламу женского товара отправили. Ограничим нашу выборку данных, только на таких людей.

In [30]:
db_w=db_sg[db_sg.womens==1]
target = ps.NominalTarget(target_selector=ps.NominalSelector('Z', 0))
# Интересует именно ошибки, поэтому True
searchSpace = ps.create_selectors(db_w, ignore=['Z',]) 
# Технически есть ещё Survived, но он нам может подсказать, в каком направлении происходит ошибка, поэтому оставляем
task = ps.SubgroupDiscoveryTask(
    db_w,
    target=target,
    search_space=searchSpace,
    result_set_size = 5, # на выход получаем 5 подгрупп
    depth = 3, # Каждая подгруппа описывается не более чем 3 предикатами
    qf = ps.ChiSquaredQF() # Функция качества на базе Chi2
)

result = ps.BeamSearch().execute(task)

for (q,sg) in result:
    print(str(q) + ":\t" + str(sg.subgroup_description))

8.351420568413513:	zip_code=Rural AND history_segment=3) $200 - $350 AND newbie=0
7.231174602632186:	recency: [2:4[ AND newbie=0 AND history_segment=1) $0 - $100
6.132738850877295:	history_segment=3) $200 - $350 AND mens=0 AND channel=Web
6.122369012591387:	zip_code=Rural AND history_segment=3) $200 - $350 AND womens=1
6.122369012591387:	zip_code=Rural AND history_segment=3) $200 - $350


## Повышаем достоверность

В данном разделе мы рассмотрим как можно повысить достоверность получаемых результатов.

Основной подход: независимая выборка.

Возьмем одну из подгрупп из предыдущего результата.

In [31]:
sg0 = result[0][1]

Провереть какие объекты из произвольной выбрки данных можно следующим образом:

In [32]:
sg0.covers(db_w).sum()

443

А посчитать статистику на этой выборке данных, можно путем создания объекта с функций качества и применением одного из её методов.

In [33]:
ps.ChiSquaredQF().evaluate_from_dataset(db_w, result[0][1])

8.351420568413513

Тогда для повышания достоверности необходимо:
* Разделить выборку на "обучающую" и тестовую выборки в соотношении 50/50
* Найти подгруппы на обучающей
* Проверить качество каждой найденной подгруппы на тестовой выборке

Реализуем этот процесс на выборке данных db_w

In [34]:
train_inds = np.random.choice(db_w.index, size = round(db_w.shape[0] * 0.5), replace = False)

db_w_train=db_w.loc[train_inds,:]
db_w_test=db_w.drop(index=train_inds)

In [35]:
target = ps.NominalTarget(target_selector=ps.NominalSelector('Z', 0))
# Интересует именно ошибки, поэтому True
searchSpace = ps.create_selectors(db_w_train, ignore=['Z',]) 
# Технически есть ещё Survived, но он нам может подсказать, в каком направлении происходит ошибка, поэтому оставляем
task = ps.SubgroupDiscoveryTask(
    db_w_train,
    target=target,
    search_space=searchSpace,
    result_set_size = 5, # на выход получаем 5 подгрупп
    depth = 3, # Каждая подгруппа описывается не более чем 3 предикатами
    qf = ps.ChiSquaredQF() # Функция качества на базе Chi2
)

result = ps.BeamSearch().execute(task)

In [36]:
qf = ps.ChiSquaredQF()
for (q,sg) in result:
    print("TRAIN: " + str(q) + ":\t" + str(sg.subgroup_description))
    print("TEST: " + str(qf.evaluate_from_dataset(db_w_test, sg)))

TRAIN: 6.5566711678499034:	zip_code=Rural AND history_segment=6) $750 - $1,000 AND mens=0
TEST: 0.15865259722250058
TRAIN: 5.80603683381694:	zip_code=Rural AND history_segment=6) $750 - $1,000 AND history>=428.89
TEST: 0.5269933724306268
TRAIN: 5.80603683381694:	zip_code=Rural AND history_segment=6) $750 - $1,000 AND newbie=1
TEST: 0.5269933724306268
TRAIN: 5.80603683381694:	zip_code=Rural AND history_segment=6) $750 - $1,000
TEST: 0.5269933724306268
TRAIN: 5.80603683381694:	zip_code=Rural AND history_segment=6) $750 - $1,000 AND womens=1
TEST: 0.5269933724306268
